In [11]:
#coding=utf-8

"""
@author: evilXu
@file: testFactor.py
@time: 2017/11/16 17:48
@description: 
"""

from rqalpha.api import *
import traceback
from datetime import *
import pymysql
import pandas as pd
from rqalpha.utils.logger import user_log


def init():
    user_log.info("test Factor init")


def compute(startdt,enddt):
    '''
    PE
    :param startdt:
    :param enddt:
    :return:
    '''
    user_log.info("testFactor compute")
    _cnnConf = {"host":"172.18.44.231","port":3306,"db":"jydb","user":"quant1","passwd":"quant1@591"}
    jyConn = pymysql.connect(host=_cnnConf.get("host"), port=_cnnConf.get("port") \
                                  , db=_cnnConf.get("db"), user=_cnnConf.get("user"),
                                  passwd=_cnnConf.get("passwd"), charset='utf8')
    _category = [1, ]
    _sectors = [1, 2, 6]
    _sql = "SELECT p.TradingDay,p.PE,a.SecuCode,a.SecuMarket" \
                   " FROM LC_DIndicesForValuation as p inner join secumain as a "\
                    "on a.innerCode=p.innerCode where a.SecuMarket in (83,90) " \
                   "and a.SecuCategory in (%s) and a.ListedSector in (%s) " \
                   "and a.ListedState!=9 and p.TradingDay between '%s' and '%s'" % (
                        ",".join([str(i) for i in _category]), ",".join([str(i) for i in _sectors])
                        ,startdt.strftime('%Y-%m-%d'),
                       enddt.strftime('%Y-%m-%d'))

    # print(_sql)
    _res = []
    _res_tmp = []
    try:
        _now = datetime.now()
        _cursor = jyConn.cursor()
        _cursor.execute(_sql)
        for _row in _cursor:
            _res_tmp.append({
                "code": _row[2] + "." + market(_row[3]),  # secumain.code(_row[0],_row[13]),#
                "date": _row[0],
                "value": float(_row[1])
            })
    except Exception as e:
        traceback.print_exc()
        return

    _res_tmp = sorted(_res_tmp,key=lambda x:x['date'])
    _res = []
    if len(_res_tmp) >0:
        _lastDt = _res_tmp[0]['date']
        _adt = {"date":_lastDt}
        for item in _res_tmp:
            if item['date'] != _lastDt:
                _res.append(_adt)
                _lastDt = item['date']
                _adt = {"date": _lastDt}
            else:
                _adt.update({item['code']:item['value']})
        if len(_adt) > 1:
            _res.append(_adt)
    if len(_res) < 1:
        return pd.DataFrame()
    return pd.DataFrame(_res).set_index(['date'])
    # print("testFactor")
                        
def market(market=90):
    if market == 83:
        return 'XSHG'
    elif market == 90:
        return 'XSHE'
    else:
        return ""

if __name__ == "__main__":
    fValue = compute(datetime(2015,6,8),datetime(2015,7,1).date())
    # _fValue_aDay = fValue.iloc[0]
    print(fValue)
    print(fValue.index)

            000001.XSHE  000002.XSHE  000004.XSHE  000005.XSHE  000006.XSHE  \
date                                                                          
2015-06-08      12.1409      11.9479     749.2220     277.9779      37.5411   
2015-06-09      11.9233          NaN     723.4597     283.1991      38.6546   
2015-06-10      11.7126      11.5318     726.6359     288.6292      37.5411   
2015-06-11      11.5722          NaN     771.1022     295.1035      37.1434   
2015-06-12      11.5932      11.5021     833.7432     293.2238      38.6148   
2015-06-15      11.1789      11.1083     796.8644     281.1106      36.3282   
2015-06-16      10.9823      10.6848     751.6923     253.7514      33.2859   
2015-06-17      11.0455      10.8334     792.2766     255.4222      34.3398   
2015-06-18      10.7997      10.5362     763.5147     246.0240      34.3994   
2015-06-19      10.2731      10.0681     694.8743     235.9992      30.9197   
2015-06-23      10.4978      10.6031     700.1679   

In [14]:
# evaluateFileDemo
from rqalpha.mod.rqalpha_mod_alphaStar_factors import evaluate_file

config = {
  "base": {
    "start_date": "2010-06-01",
    "end_date": "2016-7-01",
  },
}

factor_file_path = "./testFactor.ipynb"#"./rqalpha/examples/testFactor.py"

evaluate_file(factor_file_path=factor_file_path,config=config,config_file = "E:/evilAlpha/rqalpha/rqalpha/config_factor.yml")

haha: E:/evilAlpha/rqalpha/rqalpha/config_factor.yml


2010-06-01 INFO test Factor init
2010-06-01 INFO testFactor compute


KeyboardInterrupt: 